In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from pathlib import Path
from time import sleep

DRIVER = Path('./geckodriver').absolute()

In [2]:
URL = 'https://www.iucnredlist.org/search/list'

In [3]:
# search the "search box" by country and extract return html of main content
def search_country(country):
    search_box.clear()
    search_box.send_keys(country)
    search_button.click()
    # wait for search
    sleep(10)
    main_content = driver.find_element_by_class_name('layout-page__major')
    main_html = main_content.get_attribute('innerHTML')
    return main_html

In [4]:
# extract species info from html li items
def extract_content(html):
    results = []
    # get html with beautifulsoup
    soup = BeautifulSoup(html, 'html.parser')
    species = soup.find_all('li', class_='list-results__item')
    for s in species:
        result = {}
        result['classification'] = s.contents[0].string
        result['name'] = s.contents[2].string
        result['status'] = s.contents[3].string
        result['region'] = s.contents[4].string
        result['level'] = s.contents[5].get('title')
        results.append(result)
    return results

In [5]:
# main access to webpage
driver = webdriver.Firefox(executable_path=DRIVER)
driver.get(URL)
search_box = driver.find_element_by_class_name('search--site')
search_button = driver.find_element_by_class_name('search--site__button__key')

# test for some countrys
OECD_COUNTRYS = ['Austria', 'Germany', 'Italy']
country_results = {}
for country in OECD_COUNTRYS:
    html = search_country(country)
    country_results[country] = extract_content(html)

In [6]:
print(country_results)

{'Austria': [{'classification': 'plantae — magnoliopsida', 'name': 'Rorippa austriaca', 'status': 'Unknown', 'region': 'Global', 'level': 'Least Concern'}, {'classification': 'plantae — liliopsida', 'name': 'Gymnadenia archiducis-joannis', 'status': 'Decreasing', 'region': 'Global, Europe', 'level': 'Endangered'}, {'classification': 'plantae — liliopsida', 'name': 'Gymnadenia stiriaca', 'status': 'Decreasing', 'region': 'Global, Europe', 'level': 'Endangered'}, {'classification': 'plantae — liliopsida', 'name': 'Gymnadenia lithopolitanica', 'status': 'Decreasing', 'region': 'Global, Europe', 'level': 'Endangered'}, {'classification': 'plantae — pinopsida', 'name': 'Pinus nigra', 'status': 'Stable', 'region': 'Global', 'level': 'Least Concern'}, {'classification': 'plantae — magnoliopsida', 'name': 'Sorbus austriaca', 'status': 'Stable', 'region': 'Global, Europe', 'level': 'Least Concern'}, {'classification': 'animalia — reptilia', 'name': 'Coronella austriaca', 'status': 'Decreasing',